In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import ast
from collections import Counter
import pycountry
from datetime import datetime
import seaborn as sns
import numpy as np
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler
import sqlalchemy

In [2]:
track_genres_df = pd.read_csv("../feature_engineering/data/track_genres.csv", sep=";", header=None)
track_genres_df.columns = ["track_id", "genres"]
track_genres_df = track_genres_df[track_genres_df["genres"] != "[]"]
rel_tracks = set(track_genres_df["track_id"])
track_genres_df["track_id"].nunique()

147156

In [3]:
classification_df = pd.read_csv("classification.csv", sep=";", index_col="user_id")
classification_df.head()

,cluster
user_id,
10883488,1
35212267,3
38189090,3
22113634,3
3704198,3


In [4]:
user_track_df = pd.read_csv("../feature_engineering/data/user_track.csv", sep=";", index_col="user_id")
user_track_df.head()

C:\Users\pmuellner\AppData\Local\Continuum\anaconda3\lib\site-packages\numpy\lib\arraysetops.py:569: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  mask |= (ar1 == a)


,track_id,artist_id
user_id,,
31435741,53,21
31435741,86,32
31435741,127,44
31435741,182,57
31435741,219,65


In [5]:
user_track_df = user_track_df[user_track_df["track_id"].isin(rel_tracks)]
len(user_track_df)

1284855

In [6]:
user_class_track_df = user_track_df.merge(classification_df, left_index=True, right_index=True)
user_class_track_df.head()

,track_id,artist_id,cluster
user_id,,,
1049656,85503,14289,3
1049656,164817,2865,3
1049656,10728,1696,3
1049656,186967,1696,3
1049656,10756,1696,3


In [7]:
user_class_track_df["track_id"].nunique()

147156

In [ ]:
# tracks per user

In [ ]:
user_class_track_df.groupby(by=["cluster", "user_id"])["track_id"].nunique().groupby(by="cluster").mean()

In [ ]:
user_class_track_df.groupby(by=["cluster", "user_id"])["track_id"].nunique().groupby(by="cluster").std()

In [ ]:
%matplotlib qt
user_class_track_df.groupby(by=["cluster", "user_id"])["track_id"].nunique().to_frame().boxplot(by="cluster")

In [ ]:
# artists per user
user_class_track_df.groupby(by=["cluster", "user_id"])["artist_id"].nunique().groupby(by="cluster").mean()

In [ ]:
user_class_track_df.groupby(by=["cluster", "user_id"])["artist_id"].nunique().groupby(by="cluster").std()

In [ ]:
# artists per usergroup
#user_class_track_df.groupby(by="cluster")["artist_id"].apply(set).apply(len)
user_class_track_df.groupby(by="cluster")["artist_id"].nunique()

In [ ]:
track_genres_df = pd.read_csv("../feature_engineering/data/track_genres.csv", sep=";", header=None)
track_genres_df.columns = ["track_id", "genres"]
track_genres_df.set_index("track_id", inplace=True)
track_genres_df.head()

In [ ]:
# tracks per usergroup
#user_class_track_df.groupby(by="cluster")["artist_id"].apply(set).apply(len)
user_class_track_df.groupby(by="cluster")["track_id"].nunique()

In [ ]:
# listening events per user
events_df = pd.read_csv("../feature_engineering/data/lowms_les.csv", sep=";")
events_df.head()

In [ ]:
events_df["track_id"].nunique()

In [ ]:
events_df["user_id"].nunique()

In [ ]:
df = events_df.merge(classification_df, left_on="user_id", right_index=True)
df.groupby(by=["cluster", "user_id"]).size().groupby(by="cluster").mean()

In [ ]:
df.groupby(by=["cluster", "user_id"]).size().groupby(by="cluster").std()

In [ ]:
df.groupby("cluster").size()

In [ ]:
# listening events per usergroup

In [8]:
events_df = pd.read_csv("../feature_engineering/data/lowms_les.csv", sep=";")
events_df.head()

,user_id,track_id,artist_id,timestamp
0,31435741,53,21,1370977938
1,31435741,53,21,1370977728
2,31435741,53,21,1370977518
3,31435741,53,21,1370977308
4,31435741,53,21,1370977098


In [ ]:
print(len(events_df))
events_df = events_df[events_df["track_id"].isin(rel_tracks)]
print(len(events_df))

In [ ]:
classification_df.merge(events_df, left_index=True, right_on="user_id").groupby(by="cluster").size()

In [ ]:
classification_df.groupby(by="cluster").size()

In [ ]:
# les per track
df.head()

In [ ]:
df.groupby(by=["cluster", "track_id"]).size().groupby("cluster").mean() / classification_df.groupby(by="cluster").size()

In [ ]:
df.groupby(by=["cluster", "track_id"]).size().groupby("cluster").std()  / classification_df.groupby(by="cluster").size()

In [ ]:
# genres per user

In [ ]:
genres_per_user_df = user_track_df.merge(track_genres_df, left_on="track_id", right_index=True)#.reset_index().groupby(by="user_id")
genres_per_user_df = genres_per_user_df["genres"].apply(lambda row: ast.literal_eval(row)).to_frame()
genres_per_user_df.reset_index(inplace=True)
genres_per_user_df.head()

In [ ]:
df = genres_per_user_df.groupby(by="user_id").sum()["genres"].apply(set).to_frame()
df = classification_df.merge(df, left_index=True, right_index=True)
df["genres"] = df["genres"].apply(len)
df.groupby(by="cluster")["genres"].mean().to_frame()

In [ ]:
df.groupby(by="cluster")["genres"].std().to_frame()

In [ ]:
# mainstreaminess per usergroup

In [ ]:
mainstreaminess_df = pd.read_csv("../feature_engineering/data/low_main_users.txt", usecols=["M_global_R_APC", "user_id"], index_col="user_id")
mainstreaminess_df.head()

In [ ]:
mainstreaminess_df.merge(classification_df, left_index=True, right_index=True).groupby(by="cluster").mean()

In [ ]:
mainstreaminess_df.merge(classification_df, left_index=True, right_index=True).groupby(by="cluster").std()

In [ ]:
# user demographics

In [47]:
user_df = pd.read_csv("../feature_engineering/data/low_main_users.txt", index_col="user_id")
user_df.head()

,cnt_listeningevents,cnt_distinct_artists,novelty_artist_avg_year,cnt_distinct_tracks,cnt_listeningevents_per_week,M_global_R_APC,country,age,gender
user_id,,,,,,,,,
1049656,11698.0,792.0,0.427083154519399,3256.0,164.9830,0.096060,FI,35,m
1055118,5365.0,440.0,0.524092843135198,1404.0,15.0147,0.088300,US,34,m
1056935,8365.0,136.0,0.16214576150689805,2713.0,43.1365,0.076351,UK,30,m
1070023,14118.0,711.0,0.5249859260188209,3089.0,59.9630,0.000605,US,32,m
1072752,12749.0,484.0,0.5596268346103338,2899.0,39.7548,0.052467,DK,48,m


In [59]:
demographics_df = user_df.merge(classification_df, left_index=True, right_index=True)[["country", "age", "gender", "cluster"]]
demographics_df = demographics_df[demographics_df["age"] != -1]
demographics_df = demographics_df[demographics_df["gender"] != "n"]
demographics_df.head()

,country,age,gender,cluster
user_id,,,,
1049656,FI,35,m,3
1055118,US,34,m,2
1056935,UK,30,m,2
1070023,US,32,m,4
1072752,DK,48,m,1


In [55]:
demographics_df[["cluster", "age"]].boxplot(by="cluster")

In [49]:
demographics_df.groupby(by="cluster")["age"].mean()

cluster
1    27.951020
2    23.868307
3    31.794118
4    24.620833
Name: age, dtype: float64

In [ ]:
demographics_df.groupby(by="cluster")["age"].std()

In [60]:
gender_count_df =demographics_df.groupby(by="cluster")["gender"].value_counts()
gender_count_df

cluster  gender
1        m         154
         f          91
2        m         464
         f         227
3        m          57
         f          11
4        m         395
         f          85
Name: gender, dtype: int64

In [61]:
n_users_df = demographics_df.groupby(by="cluster")["gender"].count()
n_users_df

cluster
1    245
2    691
3     68
4    480
Name: gender, dtype: int64

In [58]:
gender_count_df / n_users_df

cluster  gender
1        m         0.587786
         f         0.347328
         n         0.064885
2        m         0.645341
         f         0.315716
         n         0.038943
3        m         0.791667
         f         0.152778
         n         0.055556
4        m         0.762548
         f         0.164093
         n         0.073359
Name: gender, dtype: float64

In [ ]:
demographics_df[["country", "cluster"]].groupby(by="cluster")["country"].value_counts().groupby(by="cluster").head(10)

In [ ]:
# time

In [9]:
events_df.head()

,user_id,track_id,artist_id,timestamp
0,31435741,53,21,1370977938
1,31435741,53,21,1370977728
2,31435741,53,21,1370977518
3,31435741,53,21,1370977308
4,31435741,53,21,1370977098


In [10]:
timestamps_df = classification_df.merge(events_df, left_index=True, right_on="user_id")[["user_id", "cluster", "timestamp"]]
timestamps_df.set_index("user_id", inplace=True)
timestamps_df.head()

,cluster,timestamp
user_id,,
10883488,1,1366331391
10883488,1,1366019650
10883488,1,1365867092
10883488,1,1364979529
10883488,1,1364979528


In [11]:
timestamps_df["timestamp"] = pd.to_datetime(timestamps_df["timestamp"], unit="s")
timestamps_df.head()

,cluster,timestamp
user_id,,
10883488,1,2013-04-19 00:29:51
10883488,1,2013-04-15 09:54:10
10883488,1,2013-04-13 15:31:32
10883488,1,2013-04-03 08:58:49
10883488,1,2013-04-03 08:58:48


In [12]:
timestamps_df["day"] = timestamps_df["timestamp"].dt.dayofweek
timestamps_df.head()

,cluster,timestamp,day
user_id,,,
10883488,1,2013-04-19 00:29:51,4
10883488,1,2013-04-15 09:54:10,0
10883488,1,2013-04-13 15:31:32,5
10883488,1,2013-04-03 08:58:49,2
10883488,1,2013-04-03 08:58:48,2


In [13]:
timestamps_df["weekend"] = timestamps_df["day"] >= 5

In [14]:
timestamps_df.head()

,cluster,timestamp,day,weekend
user_id,,,,
10883488,1,2013-04-19 00:29:51,4,False
10883488,1,2013-04-15 09:54:10,0,False
10883488,1,2013-04-13 15:31:32,5,True
10883488,1,2013-04-03 08:58:49,2,False
10883488,1,2013-04-03 08:58:48,2,False


In [ ]:
#events_weekend_df = timestamps_df.groupby(by=["cluster"])["weekend"].value_counts()
df = timestamps_df.groupby(by="user_id")["weekend"].mean().to_frame().merge(classification_df, left_index=True, right_on="user_id")
1 - df.groupby(by="cluster").mean()

In [ ]:
n_events_df = pd.DataFrame()
#n_events_df["count"] = timestamps_df.groupby(by="cluster").count()["weekend"]
#n_events_df = n_events_df.squeeze()
n_events_df = timestamps_df.groupby(by="cluster")["timestamp"].count()

In [ ]:
events_weekend_df / n_events_df

In [15]:
timestamps_df["hour"] = timestamps_df["timestamp"].dt.hour
timestamps_df.head()

,cluster,timestamp,day,weekend,hour
user_id,,,,,
10883488,1,2013-04-19 00:29:51,4,False,0
10883488,1,2013-04-15 09:54:10,0,False,9
10883488,1,2013-04-13 15:31:32,5,True,15
10883488,1,2013-04-03 08:58:49,2,False,8
10883488,1,2013-04-03 08:58:48,2,False,8


In [16]:
timestamps_df["workday"] = np.logical_and(np.logical_and(timestamps_df["hour"] <= 18, timestamps_df["hour"] >= 7), timestamps_df["day"] <= 4)
#mask = (timestamps_df["hour"] <= 18) & (timestamps_df["hour"] >= 9) & (timestamps_df["day"] <= 4)
#timestamps_df = timestamps_df.loc[mask]
timestamps_df.head()

,cluster,timestamp,day,weekend,hour,workday
user_id,,,,,,
10883488,1,2013-04-19 00:29:51,4,False,0,False
10883488,1,2013-04-15 09:54:10,0,False,9,True
10883488,1,2013-04-13 15:31:32,5,True,15,False
10883488,1,2013-04-03 08:58:49,2,False,8,True
10883488,1,2013-04-03 08:58:48,2,False,8,True


In [24]:
timestamps_df[["workday", "cluster"]].groupby(by=["cluster", "user_id"])["workday"].mean().groupby("cluster").mean()

cluster
1    0.409444
2    0.400637
3    0.434644
4    0.420096
Name: workday, dtype: float64

In [25]:
timestamps_df[["workday", "cluster"]].groupby(by=["cluster", "user_id"])["workday"].mean().groupby("cluster").std()

cluster
1    0.177590
2    0.167363
3    0.173332
4    0.163951
Name: workday, dtype: float64

In [17]:
n_events_workday_df = timestamps_df[timestamps_df["workday"]].groupby(by="user_id")["workday"].count()
n_events_workday_df.head()

user_id
1049656    3647
1055118     253
1056935    2907
1070023     110
1072752     857
Name: workday, dtype: int64

In [18]:
n_events_noworkday_df = timestamps_df[~timestamps_df["workday"]].groupby(by="user_id")["workday"].count()
n_events_noworkday_df.head()

user_id
1049656    1796
1055118     637
1056935     170
1070023     859
1072752    1099
Name: workday, dtype: int64

In [19]:
df = (n_events_workday_df / (n_events_workday_df + n_events_noworkday_df)).to_frame()
df = df.merge(classification_df, left_index=True, right_index=True)
df.groupby(by="cluster").mean()

,workday
cluster,
1,0.409444
2,0.400416
3,0.434644
4,0.420096


In [26]:
df = classification_df.copy()
df = df.merge(timestamps_df.groupby(by="user_id")["workday"].mean().to_frame(), left_index=True, right_index=True)
df.groupby(by="cluster").describe()

workday                                                              \
          count      mean       std       min       25%       50%       75%   
cluster                                                                       
1         390.0  0.409444  0.177590  0.023307  0.298339  0.402631  0.513899   
2         902.0  0.400637  0.167363  0.000000  0.275853  0.404320  0.506948   
3         101.0  0.434644  0.173332  0.015942  0.311852  0.408526  0.525664   
4         680.0  0.420096  0.163951  0.030507  0.302292  0.421797  0.516876   

                   
              max  
cluster            
1        0.980551  
2        1.000000  
3        0.893645  
4        0.960194

In [27]:
events_workday_df = timestamps_df.groupby(by=["cluster", "user_id"])["workday"].value_counts()
#events_workday_df = timestamps_df.groupby(by="cluster")["workday"].value_counts()
events_workday_df.head()

cluster  user_id  workday
1        1072752  False      1099
                  True        857
         2095434  False      1172
                  True        937
         2109450  False      1683
Name: workday, dtype: int64

In [28]:
(events_workday_df / events_workday_df.groupby(level=[1]).sum())

cluster  user_id   workday
1        1072752   False      0.561861
                   True       0.438139
         2095434   False      0.555714
                   True       0.444286
         2109450   False      0.845729
                                ...   
4        49874586  True       0.482704
         49951110  True       0.519830
                   False      0.480170
         50071075  False      0.786982
                   True       0.213018
Name: workday, Length: 4144, dtype: float64

In [29]:
events_workday_df / n_events_df

NameError: name 'n_events_df' is not defined

In [ ]:
timestamps_df["freetime"] = ~timestamps_df["workday"]
timestamps_df.head()

In [ ]:
events_freetime_df = timestamps_df.groupby(by="cluster")["freetime"].value_counts()
events_freetime_df

In [ ]:
events_freetime_df / n_events_df

In [ ]:
# user similarity

In [ ]:
genres_per_user_df.head()

In [ ]:
genre_occs_df = genres_per_user_df.merge(classification_df, left_on="user_id", right_index=True)["genres"].apply(Counter).to_frame()
genre_occs_df.index.name = "user_id"
genre_occs_df.head()

In [ ]:
track_genres_df["genres"] = track_genres_df["genres"].apply(lambda row: ast.literal_eval(row))

In [ ]:
track_genres_df.head()

In [ ]:
# life ladder per usergroup

In [ ]:
world_happiness_df = pd.read_csv("../feature_engineering/data/world_happiness_latest.csv", sep=";")
world_happiness_df.head()

In [ ]:
demographics_df.head()

In [ ]:
pycountry.countries.get(alpha_2="FI").name

In [ ]:
user_lifeladder_df = pd.DataFrame()
for uid, row in demographics_df.iterrows():
    ctr = row[0]
    cluster = row[3]
    country_name = pycountry.countries.get(alpha_2=ctr)
    if country_name:
        new_row = pd.DataFrame(index=[uid], data={"country": [country_name.name], "cluster": [cluster]})
        user_lifeladder_df = user_lifeladder_df.append(new_row)

In [ ]:
user_lifeladder_df.index.name = "user_id"
user_lifeladder_df.head()

In [ ]:
user_lifeladder_df = user_lifeladder_df.reset_index().merge(world_happiness_df, left_on="country", right_on="country")
user_lifeladder_df.head()

In [ ]:
%matplotlib qt
#%matplotlib inline 

del user_lifeladder_df["user_id"]
del user_lifeladder_df["year"]
del user_lifeladder_df["Healthy life expectancy at birth"]
user_lifeladder_df.groupby(by="cluster").boxplot()

In [ ]:
track_cluster_df = pd.read_csv("track_to_cluster.csv", sep=";", index_col="track_id")
track_cluster_df = track_cluster_df[track_cluster_df["cluster"] != -1]
track_cluster_df.head()

In [ ]:
df = track_cluster_df.merge(track_genres_df, left_index=True, right_index=True)
df.merge(events_df, left_index=True, right_on="track_id")

In [ ]:
# events per genre in cluster

In [ ]:
event_genres_df = events_df.merge(track_genres_df, left_on="track_id", right_on="track_id").merge(classification_df, left_on="user_id", right_index=True)
event_genres_df.head()

In [ ]:
event_genres_df["genres"] = event_genres_df["genres"].apply(lambda row: ast.literal_eval(row))
event_genres_df["genres"] = event_genres_df["genres"].apply(Counter)

In [ ]:
len(event_genres_df)

In [ ]:
n_tracks_per_cluster = user_track_df.merge(classification_df, left_index=True, right_on="user_id")[["track_id", "cluster"]].groupby(by="cluster")["track_id"].nunique()

In [ ]:
n_tracks_per_cluster

In [ ]:
df = event_genres_df.groupby(by="cluster")["genres"].sum() / n_tracks_per_cluster

In [ ]:
df.head()

In [ ]:
%matplotlib qt
c = 1
for count in df:
    sorted_occs = sorted(dict(count).items(), key=lambda tup: tup[1])[::-1][:30]
    ys = list(zip(*sorted_occs))[1]
    ys = np.array(ys) / n_tracks_per_cluster.loc[c]
    plt.plot(list(range(30)), ys, "-o", label="U" + str(c))
    c += 1
plt.legend()
plt.ylabel("Listening Events")
plt.xlabel("Top 30 genres")
plt.show()
    

In [ ]:
# events per workday

In [ ]:
workday_df = pd.read_csv("../feature_engineering/data/rel_events_within_workday.csv", sep=";")
workday_df.head()

In [ ]:
workday_df.merge(classification_df, left_on="user_id", right_on="user_id").groupby(by="cluster")["LEs"].mean()

In [ ]:
users_df = pd.read_csv("../feature_engineering/data/low_main_users.txt", index_col="user_id")
users_df.head()

In [ ]:
df = users_df[["cnt_listeningevents", "cnt_distinct_artists", "cnt_distinct_tracks"]].stack().to_frame()
df.columns = ["value"]
df.index.names = ["user_id", "feature"]
df = df.reset_index()
df.head()

In [ ]:
df = df.merge(classification_df, left_on="user_id", right_on="user_id")
df.set_index("user_id", inplace=True)
df.head()

In [ ]:
sns.boxplot(x="value", y="feature", hue="cluster", data=df)

In [ ]:
user_features_df = pd.DataFrame()

In [ ]:
df = events_df.groupby(by="user_id")["timestamp"].count().to_frame().merge(classification_df, left_on="user_id", right_on="user_id")
df.columns = ["user_id", "LEs", "cluster"]
df.head()

In [ ]:
user_features_df = df[["user_id", "LEs", "cluster"]].copy()

In [ ]:
df.groupby(by="cluster")["LEs"].mean()

In [ ]:
events_per_user_df = df.copy()

In [ ]:
df = events_df.groupby(by="user_id")["track_id"].nunique().to_frame().merge(classification_df, left_on="user_id", right_on="user_id")[["user_id", "track_id", "cluster"]]
df.columns = ["user_id", "n_dist_tracks", "cluster"]
df.head()

In [ ]:
tracks_per_cluster_df = pd.read_csv("track_to_cluster.csv", sep=";")
tracks_per_cluster_df = tracks_per_cluster_df.groupby(by="cluster").count()
tracks_per_cluster_df.columns = ["n_tracks"]
tracks_per_cluster_df = tracks_per_cluster_df.drop(-1)
tracks_per_cluster_df.index += 1
tracks_per_cluster_df

In [ ]:
pd.options.display.float_format = '{:20,.15f}'.format

In [ ]:
df.groupby(by="cluster")["n_dist_tracks"].mean()

In [ ]:
dist_tracks_per_user_df = df.copy()

In [ ]:
events_per_user_df.head()

In [ ]:
dist_tracks_per_user_df.head()

In [ ]:
df = events_per_user_df.merge(dist_tracks_per_user_df[["user_id", "n_dist_tracks"]], left_on="user_id", right_on="user_id")
df.head()

In [ ]:
df["events_per_track"] = df["LEs"] / df["n_dist_tracks"]
df.head()

In [ ]:
df.groupby(by="cluster")["events_per_track"].mean()

In [ ]:
events_df.head()

In [ ]:
df = events_df.groupby(by="user_id")["artist_id"].nunique().to_frame().merge(classification_df, left_on="user_id", right_on="user_id")[["user_id", "artist_id", "cluster"]]
df.columns = ["user_id", "n_dist_artists", "cluster"]
df.head()

In [ ]:
user_features_df["n_dist_artists"] = user_features_df.merge(df, left_on="user_id", right_on="user_id")["n_dist_artists"]

In [ ]:
df.groupby(by="cluster")["n_dist_artists"].mean()

In [ ]:
user_features_df.head()

In [ ]:
df = user_features_df.set_index("user_id")[["LEs", "n_dist_tracks", "n_dist_artists"]].stack()
df = df.to_frame().reset_index()
df = df.merge(classification_df, left_on="user_id", right_on="user_id")

In [ ]:
df.columns = ["user_id", "feature", "value", "cluster"]
df.head()

In [ ]:
sns.boxplot(x="feature", y="value", hue="cluster", data=df, showmeans=True)

In [ ]:
df = events_df.groupby(by="user_id")["artist_id"].nunique().to_frame().merge(classification_df, left_index=True, right_on="user_id").set_index("user_id")
df.groupby(by="cluster").mean()

In [ ]:
df = events_df.groupby(by="user_id")["track_id"].nunique().to_frame().merge(classification_df, left_index=True, right_on="user_id").set_index("user_id")
df.groupby(by="cluster").mean()

In [ ]:
user_track_df["track_id"].nunique()

In [ ]:
# tracks

In [ ]:
user_track_df.merge(classification_df, left_index=True, right_on="user_id").groupby(by="cluster")["track_id"].nunique()

In [30]:
# user activity
n_events_df = events_df.groupby(by="user_id")["timestamp"].count().to_frame().merge(classification_df, left_on="user_id", right_on="user_id")
n_events_df.columns = ["LEs", "cluster"]
n_events_df.head()

,LEs,cluster
user_id,,
1049656,5443,3
1055118,890,2
1056935,3077,2
1070023,969,4
1072752,1956,1


In [31]:
from datetime import datetime
start = pd.Timestamp(datetime.fromtimestamp(events_df["timestamp"].min()))
end = pd.Timestamp(datetime.fromtimestamp(events_df["timestamp"].max()))
(start, end)

(Timestamp('2005-02-14 03:03:17'), Timestamp('2014-09-02 01:56:08'))

In [32]:
df = events_df.groupby(by="user_id")["timestamp"].agg(["min", "max"])
df["min"] = df["min"].apply(lambda r: pd.Timestamp(datetime.fromtimestamp(r)))
df["max"] = df["max"].apply(lambda r: pd.Timestamp(datetime.fromtimestamp(r)))
time_active_df = pd.DataFrame(index=df.index, data={"time": df["max"] - df["min"]})
time_active_df.head()

,time
user_id,
1049656,496 days 08:43:25
1055118,2479 days 06:37:17
1056935,1357 days 01:15:12
1070023,1580 days 01:22:13
1072752,2239 days 20:44:26


In [33]:
SQL_CREDENTIALS = "root:1234"
engine = sqlalchemy.create_engine('mysql+pymysql://' + SQL_CREDENTIALS + '@localhost:3306/music_recommender_db')

In [34]:
statement = "SELECT user_id, FROM_UNIXTIME(registered_unixtime) AS registered_time FROM users WHERE user_id IN " + str(tuple(set(df.index)))
registered_time_df = pd.read_sql(sql=statement, con=engine)

C:\Users\pmuellner\AppData\Local\Continuum\anaconda3\lib\site-packages\pymysql\cursors.py:170: Warning: (1366, "Incorrect string value: '\\xE4ische...' for column 'VARIABLE_VALUE' at row 1")
  result = self._query(query)


In [35]:
n_events_df.head()

,LEs,cluster
user_id,,
1049656,5443,3
1055118,890,2
1056935,3077,2
1070023,969,4
1072752,1956,1


In [36]:
user_activity_df = time_active_df.merge(n_events_df, left_index=True, right_index=True)
user_activity_df.head()

,time,LEs,cluster
user_id,,,
1049656,496 days 08:43:25,5443,3
1055118,2479 days 06:37:17,890,2
1056935,1357 days 01:15:12,3077,2
1070023,1580 days 01:22:13,969,4
1072752,2239 days 20:44:26,1956,1


In [37]:
user_activity_df["LEs_per_week"] = user_activity_df["LEs"] / (user_activity_df["time"].dt.days / 7)
user_activity_df.head()

,time,LEs,cluster,LEs_per_week
user_id,,,,
1049656,496 days 08:43:25,5443,3,76.816532
1055118,2479 days 06:37:17,890,2,2.513110
1056935,1357 days 01:15:12,3077,2,15.872513
1070023,1580 days 01:22:13,969,4,4.293038
1072752,2239 days 20:44:26,1956,1,6.115230


In [38]:
user_activity_df.replace([-np.inf, np.inf], np.nan, inplace=True)
user_activity_df.dropna(inplace=True)

In [46]:
%matplotlib qt
user_activity_df[["cluster", "LEs_per_week"]].boxplot(by="cluster")

In [42]:
(user_activity_df["LEs_per_week"] > 600).sum()

1

In [43]:
user_activity_df[user_activity_df["LEs_per_week"] < 600].groupby(by="cluster")["LEs_per_week"].mean()

cluster
1    18.013751
2    25.401069
3    17.803559
4    24.428883
Name: LEs_per_week, dtype: float64

In [44]:
user_activity_df[user_activity_df["LEs_per_week"] < 600].groupby(by="cluster")["LEs_per_week"].std()

cluster
1    20.210532
2    30.746741
3    20.829406
4    31.614987
Name: LEs_per_week, dtype: float64

In [ ]:
user_activity_df.groupby(by="cluster")["LEs_per_week"].mean()

In [ ]:
user_activity_df.groupby(by="cluster")["LEs_per_week"].std()

In [ ]:
user_activity_df["days_active"] = user_activity_df["time"].apply(lambda r: r.days)

In [ ]:
user_activity_df.head()

In [ ]:
user_activity_df.groupby(by="cluster")["days_active"].mean()

In [ ]:
# genres per track

In [ ]:
listened_tracks_per_cluster_df = user_track_df.merge(track_genres_df, left_on="track_id", right_index=True).merge(classification_df, left_index=True, right_on="user_id")
listened_tracks_per_cluster_df["genres"] = listened_tracks_per_cluster_df["genres"].apply(lambda r: ast.literal_eval(r))
listened_tracks_per_cluster_df.head()

In [ ]:
n_genres_per_cluster_df = listened_tracks_per_cluster_df.groupby(by=["cluster", "track_id"])["genres"].sum().apply(set).apply(len)
n_genres_per_cluster_df.head()

In [ ]:
n_genres_per_cluster_df.groupby(by="cluster").mean()

In [ ]:
n_genres_per_cluster_df.groupby(by="cluster").std()

In [ ]:
len(listened_tracks_per_cluster_df)